In [1]:
from modelArchitecture.ResnetRnnDense import ResnetRnnDense
from modelArchitecture.VirtualBatchNormalization import VirtualBatchNormalization
from modelArchitecture.Attention import Attention
from keras.models import model_from_yaml, model_from_json
from utils import split_data, translate, save_model, standardize, result
from HydraMethod import HydraMethod
import keras
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
from keras.losses import mean_absolute_error
import math
import numpy as np
from keras.backend import eval

/home/markus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
ls models

LRF_model.h5    all_data.h5    best_model/   hydra_new.yaml  hydra_new_C.yaml
LRF_model.json  all_data.yaml  hydra_new.h5  hydra_new_C.h5


In [3]:
yaml_file = open('models/all_data.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()

model = model_from_yaml(loaded_model_yaml,
                            custom_objects={'VirtualBatchNormalization':
                                            VirtualBatchNormalization,
                                            'Attention': Attention})
model.load_weights('models/all_data.h5')

In [4]:
ls data/non_seen_pep

irt_test          non_seen_pep.csv           non_seen_pep_datafram_2.csv
irt_train         non_seen_pep.npy           seq_test
non_seen_irt.npy  non_seen_pep_datafram.csv  seq_train


In [12]:
PATH = "../data/non_seen_pep/non_seen_pep_datafram.csv"
splitter = split_data(PATH, PATH, train_size=0.99)
x_train, y_train = splitter.split()



15678


In [13]:
len(x_train)

15678

In [14]:
translater = translate()

x_train = translater.translate(x_train)


y_train_std = standardize(y_train)

In [15]:
hydra = HydraMethod(ResnetRnnDense(), 4)

In [16]:
len(x_train)

15678

In [17]:
pred = model.predict(hydra.get_all_representation(x_train))

In [18]:
n = int(len(y_train_std)*0.95)
dif = abs(y_train_std.flatten() - pred.flatten())
sort_dif = np.sort(dif.flatten())
pred = 2 * sort_dif[n] /  (max(y_train_std) - min(y_train_std)) * 100
print(pred)

[23.75797401]


In [ ]:
from modelArchitecture.ResnetRnnDense import ResnetRnnDense
from modelArchitecture.VirtualBatchNormalization import VirtualBatchNormalization
from modelArchitecture.Attention import Attention
from keras.models import model_from_yaml
from utils import split_data, translate, save_model, standardize, result
from HydraMethod import HydraMethod
import keras
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
from keras.losses import mean_squared_error
import math
import numpy as np
from keras.backend import eval

PATH = "./data/"
splitter = split_data(PATH + "raw.csv", PATH, test_size=0.1, train_size=0.9)
x_train, y_train, x_test, y_test = splitter.split()
translater = translate()

x_train = translater.translate(x_train)
x_test = translater.translate(x_test)

y_train_std, y_test_std = standardize(y_train, y_test)

In [ ]:
hydra = HydraMethod(ResnetRnnDense(blocks=[2,2,2,2],features=32, ResnetDrop=0.3),4); hydra.compile()

In [ ]:
x_train, x_test = hydra.get_all_representation(x_train), hydra.get_all_representation(x_test)

In [ ]:
from utils import LRFinder
lr_finder = LRFinder(hydra.get_model())
lr_finder.find(x_train, y_train_std, 0.000001, 1, 256, 3,fit=True)

In [ ]:
lr_finder.plot_loss()

In [ ]:
from keras.models import model_from_json
model = hydra.get_model()
# serialize model to YAML
model_yaml = model.to_yaml()
with open("models/LRF_model.json", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("models/LRF_model.h5")
print("Saved model to disk")

In [ ]:
class LR():
    def __init__(self, lr):
        self.lr = lr

    def step_decay(self, epoch):
        initial_lrate = self.lr
        drop = 0.4
        epochs_drop = 10.0
        lrate = initial_lrate * math.pow(drop,
                                         math.floor((1+epoch)/epochs_drop))
        print(lrate)
        return lrate

In [ ]:
def freeze_it(model, block_nr):
    counter = 0
    for layer in model.layers:
        if "add" in layer.name:
            counter += 1
        if counter >= 4 * block_nr:
            layer.trainable = True
        else:
            layer.trainable = False
    print(counter / 4)
    return model

In [ ]:
def T95(model, x, y):
    predict_train = model.predict(x)
    res_train = result(y, predict_train)
    print(np.round(res_train.t95()[0], 4))

In [ ]:
EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              min_delta=0, patience=6,
                                              verbose=1,
                                              restore_best_weights=True)

In [ ]:
def training(model, X_train, y_train_std, X_test, y_test_std, lr, bz, k=None):
    if k:
        print("not none")
        model = freeze_it(model, k)

    lr = LR(lr)
    lrate = LearningRateScheduler(lr.step_decay)

    model.compile(loss=mean_squared_error, optimizer=Adam(7e-5))
    history = model.fit(X_train, y_train_std, validation_data=(X_test, y_test_std),
              epochs=2, batch_size=bz, callbacks=[lrate, EarlyStopping])



    T95(model,X_train,y_train_std)
    T95(model,X_test,y_test_std)
    return model


In [ ]:
from keras.models import model_from_json

yaml_file = open('models/LRF_model.json', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()

model = model_from_json(loaded_model_yaml,
                        custom_objects={'VirtualBatchNormalization':VirtualBatchNormalization, 'Attention':Attention})

model.load_weights('models/LRF_model.h5')

In [ ]:
lr = 5e-4
model = training(model,x_train,y_train_std,x_test,y_test_std,lr,256)

In [ ]:
#lr = eval(model.optimizer.lr) * 0.6
model = training(model,x_train,y_train_std,x_test,y_test_std,lr,512,6)